In [1]:
import pyedflib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import datetime

In [3]:
f = pyedflib.EdfReader(r"D:\BaiduNetdiskDownload\sz1.edf")
n = f.signals_in_file
signal_labels = f.getSignalLabels()
record = np.zeros((n, f.getNSamples()[0]))
for i in np.arange(n):
    record[i, :] = f.readSignal(i)

In [26]:
def Q_P(x,y,bins):
    '''
    x,y 代表两个信号
    bins代表区间
    该函数返回每个区间的Q、P值
    '''
    sxy=pd.DataFrame({'x':x,'y':y})
    #sxy=sxy.sort_values(by='x')
    #sxy.index=range(len(sxy))
    sxy['bins']=pd.cut(sxy['x'],bins,labels=range(1,len(bins)),include_lowest=True)
    Q=[]
    P=[]
    for i in range(len(bins)-1):
        Q.append(sxy[sxy.bins==i+1]['y'].mean())
        P.append((bins[i]+bins[i+1])/2) 
    return Q,P

def sx(i,x,P):
    '''
    i的取值范围是[1,len(P)-1]
    '''
    lp=len(P)
    if(i==1):
        if x<=P[1]:
            return 1
        else:
            return 0
    if(i==lp-1):
        if x>P[lp-2]:
            return 1
        else:
            return 0
    if P[i-1]<x<=P[i]:
        return 1
    else:
        return 0
    
def gx(i,x,Q,P):
    '''
    i的取值范围是[1,len(P)-1]
    '''
    k=(Q[i]-Q[i-1])/(P[i]-P[i-1])
    gx=k*(x-P[i-1])+Q[i-1]
    return gx

def fx(x,Q,P):
    '''
    L: the number of bins
    '''
    L=len(P)
    fx=0
    for i in range(L-1):
        fx+=sx(i+1,x,P)*gx(i+1,x,Q,P)
    return fx   

def h2(y,f):
    my=np.mean(y)
    sst=np.sum((y-my)**2)
    sse=np.sum((y-f)**2)
    h2=1-sse/sst
    return h2

In [27]:
def H2(s1,s2,L=7):
    bins1=np.linspace(np.min(s1),np.max(s1),L)
    Q1,P1=Q_P(s1,s2,bins1)
    f1=[]
    
    for x in s1:
        f1.append(fx(x,Q1,P1))
    f1=np.array(f1)
    h2s1=h2(s2,f1)
    
    bins2=np.linspace(np.min(s2),np.max(s2),L)
    Q2,P2=Q_P(s2,s1,bins2)
    f2=[]
    for x in s2:
        f2.append(fx(x,Q2,P2))
    f2=np.array(f2)
    h2s2=h2(s1,f2)
    
    s_h2 = [h2s1,h2s2]
    #H2=np.array([[0,h2s1],[h2s2,0]])
    #PQ=[[P1,Q1],[P2,Q2]]
    #return H2,PQ
    return s_h2


def H2_xy(sx,sy,L=7):
    bins = np.linspace(np.min(sx),np.max(sx),L)
    Q,P =Q_P(sx,sy,bins)
    f=[]
    for x in sx:
        f.append(fx(x,Q,P))
    f=np.array(f)
    h2_xy=h2(sy,f)
    #PQ=[P,Q]
    #return h2_xy,PQ
    return h2_xy

In [6]:
def H2_whole3(signals,start_time,duration,sampleFreq,slideWindow=2,step=1,maxlag=0.1,L=7):
    '''
    版本3：不求平均，穷举
    signals：原始信号，numpy矩阵类型（二维矩阵）
    slideWindows:滑动窗口
    duration：信号的持续时间
    '''
    time_flexity=1
    signals_num = len(signals)
    if signals_num < 2:
        print("Please input at least 2 signals to compute h2!")
        return None,None
    
    h2_num = int((duration-slideWindow-maxlag)//step + 1)
    if h2_num <= 0:
        print("Duration is too short to compute h2!")
        return None,None
    
    
    h2_value = []
    lag_value = []
    
     # 计算每次窗口滑动时的H2（每个时间段）
    for ti in range(h2_num):
        start_time_window = start_time + ti*step
        stop_time_window = start_time_window + slideWindow
        start_samples = int(start_time_window*sampleFreq)
        stop_samples = int(stop_time_window*sampleFreq)
        # 对角线上的值设置为 0
        signals_h2 = np.zeros((signals_num,signals_num))
        signals_lag = np.zeros((signals_num,signals_num))
        # 计算每两个信号之间的H2: (0,1)，(0,2)，...，(0,n-1)；(1,2)，(1,3)，...，(1,n-1)；...(n-2,n-1)
        for si1 in range(signals_num-1):
            s1 = signals[si1][start_samples:stop_samples]
            # [s1] = setSignaltoZero([signals[si1][start_samples:stop_samples]]) # 做归0处理
            for si2 in range(si1+1,signals_num):
                s2 = signals[si2][start_samples:stop_samples]
                #[s2] = setSignaltoZero([signals[si2][start_samples:stop_samples]])
                # 设置两个信号之间的H2的初始值为0
                two_signals_h2 = np.zeros(2) 
                two_signals_lag = np.zeros(2) 
                
                h2_00 = H2(s1,s2,L)
                two_signals_h2[0] = h2_00[0]
                two_signals_h2[1] = h2_00[1]
                # 考虑时间延迟
                # 计算x到y
                sample_maxlag = int(maxlag*sampleFreq)
                for tagi in range(1,sample_maxlag): # x向右移动，符号为正；y向右移动，符号为负
                    #print(start_samples,stop_samples)
                    time_flexity += 1
                    s1m = signals[si1][start_samples+tagi:stop_samples+tagi]
                    s2m = signals[si2][start_samples+tagi:stop_samples+tagi]
                    #[s1m,s2m] = setSignaltoZero([s1m,s2m],1)  # 做归0处理
                    
                    h2_mn = H2(s1m,s2,L) # s1向右移
                    h2_nm = H2(s1,s2m,L) # s2向右移
                    
                    if (h2_mn[0]>two_signals_h2[0])&(h2_mn[0]>h2_nm[0]):
                        two_signals_h2[0] = h2_mn[0]
                        two_signals_lag[0] = tagi
                        
                    elif h2_nm[0]>two_signals_h2[0]:
                        two_signals_h2[0] = h2_nm[0]
                        two_signals_lag[0] = -tagi
                        
                    if (h2_mn[1]>two_signals_h2[1])&(h2_mn[1]>h2_nm[0]):
                        two_signals_h2[1] = h2_mn[1]
                        two_signals_lag[1] = -tagi
                        
                    elif h2_nm[1]>two_signals_h2[1]:
                        two_signals_h2[1] = h2_nm[1]
                        two_signals_lag[1] = tagi
                
                signals_h2[si1,si2] = two_signals_h2[0]
                signals_h2[si2,si1] = two_signals_h2[1]
                signals_lag[si1,si2] = two_signals_lag[0]
                signals_lag[si2,si1] = two_signals_lag[1]
                
        h2_value.append(signals_h2)
        lag_value.append(signals_lag)
    
    print(time_flexity)
    return h2_value,lag_value

In [7]:
def H2_whole_dichotomy2(signals,start_time,duration,sampleFreq,slideWindow=2,step=1,maxlag=0.1,L=7):
    '''
    版本6：不求平均,lag算法改进,采用二分法计算最大h2，初始采用7点采样（而不是三点）
    signals：原始信号，numpy矩阵类型（二维矩阵）
    slideWindows:滑动窗口
    duration：信号的持续时间
    '''
    signals_num = len(signals)
    if signals_num < 2:
        print("Please input at least 2 signals to compute h2!")
        return None,None
    
    h2_num = int((duration-slideWindow-maxlag)//step + 1)
    if h2_num <= 0:
        print("Duration is too short to compute h2!")
        return None,None
    
    if maxlag <= 0:
        print("Time maxlag must be greater than zero!")
        
    h2_value = []
    lag_value = []
    dict_sample = dict()
     # 计算每次窗口滑动时的H2（每个时间段）
    for ti in range(h2_num):
        start_time_window = start_time + ti*step
        stop_time_window = start_time_window + slideWindow
        start_samples = int(start_time_window*sampleFreq)
        stop_samples = int(stop_time_window*sampleFreq)
        # 对角线上的值设置为 0
        signals_h2 = np.zeros((signals_num,signals_num))
        signals_lag = np.zeros((signals_num,signals_num))
        # 计算每两个信号之间的H2: (0,1)，(0,2)，...，(0,n-1)；(1,2)，(1,3)，...，(1,n-1)；...(n-2,n-1)
        for si1 in range(signals_num-1):
            s1 = signals[si1][start_samples:stop_samples]
            # [s1] = setSignaltoZero([signals[si1][start_samples:stop_samples]]) # 做归0处理
            
            for si2 in range(si1+1,signals_num):
                s2 = signals[si2][start_samples:stop_samples]
                #[s2] = setSignaltoZero([signals[si2][start_samples:stop_samples]])
                
                # 设置两个信号之间的H2的初始值为0
                two_signals_h2 = np.zeros(2) 
                two_signals_lag = np.zeros(2) 
                
                h2_00 = H2(s1,s2,L)
                
                two_signals_h2[0] = h2_00[0]
                two_signals_h2[1] = h2_00[1]
                # 考虑时间延迟
                # 计算x到y
                
                maxlag_points = int(maxlag*sampleFreq) # 总共样本点
                sample_num = int(maxlag_points**0.5-maxlag_points**0.25)
                if sample_num < 3:
                    sample_num = 3
                sample_points = np.percentile(range(0,maxlag_points),np.linspace(0,100,sample_num),interpolation='nearest')
                
                t0_h2 = []
                t1_h2 = []
                dict_sample.clear()
                sign = 1 # 符号位，判断是x移动还是y移动
                left = right = new_left = new_median = new_right = 0
                
                for tagi in sample_points[1:]: # x向右移动，符号为正；y向右移动，符号为负
                    
                    s1m = signals[si1][start_samples+tagi:stop_samples+tagi]
                    s2m = signals[si2][start_samples+tagi:stop_samples+tagi]

                    h2_mn = H2(s1m,s2,L) # s1向右移
                    h2_nm = H2(s1,s2m,L) # s2向右移

                    t0_h2.extend([h2_mn[0],h2_nm[0]])
                    t1_h2.extend([h2_mn[1],h2_nm[1]])
                
                t0_h2_max = max(t0_h2)
                t1_h2_max = max(t1_h2)
                
                t0_index = t0_h2.index(t0_h2_max)
                t1_index = t1_h2.index(t1_h2_max)
                
                sign = (-1)**t0_index
                if t0_h2_max>two_signals_h2[0]:
                    two_signals_h2[0] = t0_h2_max
                    sub0_h2 = t0_index//2 + 1
                    two_signals_lag[0] = sign*sample_points[sub0_h2]
                    if sub0_h2 < (sample_num - 1):
                        left = sample_points[sub0_h2-1]
                        right = sample_points[sub0_h2+1]
                        new_median = sample_points[sub0_h2]
                        new_left = (left+new_median)//2
                        new_right = (right+new_median)//2
                        #dict_sample[left] = t0_h2[t0_index-2]
                        dict_sample[new_median] = t0_h2[t0_index]
                        #dict_sample[right] = t0_h2[t0_index+2]
                    else:
                        left = sample_points[sub0_h2-2]
                        new_right = right = sample_points[sub0_h2]
                        new_median = (sample_points[sub0_h2]+sample_points[sub0_h2-1])//2
                        new_left = (sample_points[sub0_h2-1]+sample_points[sub0_h2-2])//2
                        dict_sample[sample_points[sub0_h2-1]] = t0_h2[t0_index-2]
                        dict_sample[right] = t0_h2[t0_index]
                else:
                    right = sample_points[2]
                    new_left = left = 0
                    new_right = (right+sample_points[1])//2
                    new_median = sample_points[1]//2
                    dict_sample[sample_points[1]] = t0_h2[(1-sign)//2]
                    dict_sample[0] = two_signals_h2[0]
                
                #print(sign,t0_index,sample_points[t0_index//2 + 1],left,right,new_left,new_median,new_right)
                while (new_right-new_left) >= 2:
                    t0_h2 = []
                    if dict_sample.get(new_left):
                        t0_h2.append(dict_sample[new_left])
                    else:
                        if sign>0: # 说明sx向右移
                            sxx = signals[si1][start_samples+new_left:stop_samples+new_left]
                            t0_h2.append(H2_xy(sxx,s2,L))
                        else:
                            syy = signals[si2][start_samples+new_left:stop_samples+new_left]
                            t0_h2.append(H2_xy(s1,syy,L))
                        dict_sample[new_left] = t0_h2[0]
                            
                    if dict_sample.get(new_median):
                        t0_h2.append(dict_sample[new_median])
                    else:
                        if sign>0: # 说明sx向右移
                            sxx = signals[si1][start_samples+new_median:stop_samples+new_median]
                            t0_h2.append(H2_xy(sxx,s2,L))
                        else:
                            syy = signals[si2][start_samples+new_median:stop_samples+new_median]
                            t0_h2.append(H2_xy(s1,syy,L))
                        dict_sample[new_median] = t0_h2[1]
                        
                    if dict_sample.get(new_right):
                        t0_h2.append(dict_sample[new_right])
                    else:
                        if sign>0: # 说明sx向右移
                            sxx = signals[si1][start_samples+new_right:stop_samples+new_right]
                            t0_h2.append(H2_xy(sxx,s2,L))
                        else:
                            syy = signals[si2][start_samples+new_right:stop_samples+new_right]
                            t0_h2.append(H2_xy(s1,syy,L))
                        dict_sample[new_right] = t0_h2[2]
                            
                    two_signals_h2[0] = max(t0_h2)
                    t0_index = t0_h2.index(two_signals_h2[0])
                    
                    if t0_index == 1:
                        two_signals_lag[0] = sign*new_median
                        left = new_left
                        right = new_right
                        new_left = (new_left+new_median)//2
                        new_right = (new_median+new_right)//2
                    elif t0_index == 0:
                        two_signals_lag[0] = sign*new_left
                        if left==new_left:
                            right = new_right = new_median
                            new_median = (new_right+new_left)//2
                        else:
                            right = new_median
                            new_right = (new_median+new_left)//2
                            new_median = new_left
                            new_left = (left+new_left)//2
                    else:
                        two_signals_lag[0] = sign*new_right
                        if right==new_right:
                            left = new_left = new_median
                            new_median = (new_right+new_left)//2
                        else:
                            left = new_median
                            new_left = (new_median+new_right)//2
                            new_median = new_right
                            new_right = (new_right+right)//2
                    
                sign = (-1)**(t1_index+1)
                dict_sample.clear()
                if t1_h2_max>two_signals_h2[1]:
                    two_signals_h2[1] = t1_h2_max
                    sub1_h2 = t1_index//2 + 1
                    two_signals_lag[1] = sign*sample_points[sub1_h2]
                    if sub1_h2 < (sample_num - 1):
                        left = sample_points[sub1_h2-1]
                        right = sample_points[sub1_h2+1]
                        new_median = sample_points[sub1_h2]
                        new_left = (left+new_median)//2
                        new_right = (right+new_median)//2
                        #dict_sample[left] = t0_h2[t0_index-2]
                        dict_sample[new_median] = t1_h2[t1_index]
                        #dict_sample[right] = t0_h2[t0_index+2]
                    else:
                        left = sample_points[sub1_h2-2]
                        new_right = right = sample_points[sub1_h2]
                        new_median = (sample_points[sub1_h2]+sample_points[sub1_h2-1])//2
                        new_left = (sample_points[sub1_h2-1]+sample_points[sub1_h2-2])//2
                        dict_sample[sample_points[sub1_h2-1]] = t1_h2[t1_index-2]
                        dict_sample[right] = t1_h2[t1_index]
                else:
                    right = sample_points[2]
                    new_left = left = 0
                    new_right = (right+sample_points[1])//2
                    new_median = sample_points[1]//2
                    dict_sample[sample_points[1]] = t1_h2[(1+sign)//2]
                    dict_sample[0] = two_signals_h2[1]
                
                #print(sign,t0_index,sample_points[t0_index//2 + 1],left,right,new_left,new_median,new_right)
                while (new_right-new_left) >= 2:
                    t1_h2 = []
                    if dict_sample.get(new_left):
                        t1_h2.append(dict_sample[new_left])
                    else:
                        if sign<0: # 说明sx向右移
                            sxx = signals[si1][start_samples+new_left:stop_samples+new_left]
                            t1_h2.append(H2_xy(s2,sxx,L))
                        else:
                            syy = signals[si2][start_samples+new_left:stop_samples+new_left]
                            t1_h2.append(H2_xy(syy,s1,L))
                        dict_sample[new_left] = t1_h2[0]
                            
                    if dict_sample.get(new_median):
                        t1_h2.append(dict_sample[new_median])
                    else:
                        if sign<0: # 说明sx向右移
                            sxx = signals[si1][start_samples+new_median:stop_samples+new_median]
                            t1_h2.append(H2_xy(s2,sxx,L))
                        else:
                            syy = signals[si2][start_samples+new_median:stop_samples+new_median]
                            t1_h2.append(H2_xy(syy,s1,L))
                        dict_sample[new_median] = t1_h2[1]
                        
                    if dict_sample.get(new_right):
                        t1_h2.append(dict_sample[new_right])
                    else:
                        if sign<0: # 说明sx向右移
                            sxx = signals[si1][start_samples+new_right:stop_samples+new_right]
                            t1_h2.append(H2_xy(s2,sxx,L))
                        else:
                            syy = signals[si2][start_samples+new_right:stop_samples+new_right]
                            t1_h2.append(H2_xy(syy,s1,L))
                        dict_sample[new_right] = t1_h2[2]
                            
                    two_signals_h2[1] = max(t1_h2)
                    t1_index = t1_h2.index(two_signals_h2[1])
                    
                    if t1_index == 1:
                        two_signals_lag[1] = sign*new_median
                        left = new_left
                        right = new_right
                        new_left = (new_left+new_median)//2
                        new_right = (new_median+new_right)//2
                    elif t1_index == 0:
                        two_signals_lag[1] = sign*new_left
                        if left==new_left:
                            right = new_right = new_median
                            new_median = (new_right+new_left)//2
                        else:
                            right = new_median
                            new_right = (new_median+new_left)//2
                            new_median = new_left
                            new_left = (left+new_left)//2
                    else:
                        two_signals_lag[1] = sign*new_right
                        if right==new_right:
                            left = new_left = new_median
                            new_median = (new_right+new_left)//2
                        else:
                            left = new_median
                            new_left = (new_median+new_right)//2
                            new_median = new_right
                            new_right = (new_right+right)//2
                            
                    #print(t0_index,left,right,new_left,new_median,new_right)        
                #print(dict_sample)
                
                signals_h2[si1,si2] = two_signals_h2[0]
                signals_h2[si2,si1] = two_signals_h2[1]
                signals_lag[si1,si2] = two_signals_lag[0]
                signals_lag[si2,si1] = two_signals_lag[1]
                
        h2_value.append(signals_h2)
        lag_value.append(signals_lag)
    
    #print(time_flexity)
    return h2_value,lag_value

In [10]:
def print_h2_lag(h2_value,lag_value):
    for i,h2_i,lag_i in zip(range(len(h2_value)),h2_value,lag_value):
        print(i+1,"：",lag_i[0],lag_i[1],'\n',h2_i)
        print()

In [8]:
signals1 = np.array([record[56],record[66]])
signals2 = np.array([record[46],record[56]])
signals3 = np.array([record[46],record[56],record[66]])

In [9]:
t_start = datetime.datetime.now()
h2_value1,lag_value1 = H2_whole_dichotomy2(signals2,0,12,1000,slideWindow=2,step=1,maxlag=0.2)
t_end = datetime.datetime.now()
print(t_end-t_start)

0:00:24.676943


In [11]:
print_h2_lag(h2_value1,lag_value1)

1 ： [   0. -190.] [190.   0.] 
 [[0.         0.31846365]
 [0.32894879 0.        ]]

2 ： [  0. -46.] [30.  0.] 
 [[0.         0.38352   ]
 [0.37726124 0.        ]]

3 ： [  0. -90.] [130.   0.] 
 [[0.         0.3111908 ]
 [0.34930989 0.        ]]

4 ： [   0. -110.] [-100.    0.] 
 [[0.         0.19811423]
 [0.2063928  0.        ]]

5 ： [  0. -20.] [190.   0.] 
 [[0.         0.17348006]
 [0.14632529 0.        ]]

6 ： [ 0. 60.] [108.   0.] 
 [[0.         0.22262534]
 [0.11265209 0.        ]]

7 ： [  0. 110.] [199.   0.] 
 [[0.         0.12606271]
 [0.10691637 0.        ]]

8 ： [  0. 199.] [-199.    0.] 
 [[0.         0.19457846]
 [0.22589168 0.        ]]

9 ： [0. 0.] [-180.    0.] 
 [[0.         0.32251269]
 [0.33526995 0.        ]]

10 ： [0. 0.] [0. 0.] 
 [[0.         0.39882362]
 [0.40902538 0.        ]]



In [12]:
t_start = datetime.datetime.now()
h2_value2,lag_value2 = H2_whole_dichotomy2(signals3,0,12,1000,slideWindow=2,step=1,maxlag=0.2)
t_end = datetime.datetime.now()
print(t_end-t_start)

0:01:11.725670


In [13]:
print_h2_lag(h2_value2,lag_value2)

1 ： [   0. -190.  -20.] [190.   0. 150.] 
 [[0.         0.31846365 0.25939455]
 [0.32894879 0.         0.33884469]
 [0.19353317 0.30760651 0.        ]]

2 ： [   0.  -46. -199.] [  30.    0. -199.] 
 [[0.         0.38352    0.2530617 ]
 [0.37726124 0.         0.59168956]
 [0.23374683 0.67084149 0.        ]]

3 ： [   0.  -90. -166.] [ 130.    0. -199.] 
 [[0.         0.3111908  0.23718602]
 [0.34930989 0.         0.58463159]
 [0.26544084 0.60215684 0.        ]]

4 ： [   0. -110.   20.] [-100.    0.    0.] 
 [[0.         0.19811423 0.162487  ]
 [0.2063928  0.         0.39980622]
 [0.17110855 0.40066686 0.        ]]

5 ： [  0. -20. -10.] [190.   0.   0.] 
 [[0.         0.17348006 0.33169454]
 [0.14632529 0.         0.6113036 ]
 [0.18691541 0.60543047 0.        ]]

6 ： [ 0. 60. 90.] [108.   0.   0.] 
 [[0.         0.22262534 0.37924723]
 [0.11265209 0.         0.67042045]
 [0.2943878  0.61163598 0.        ]]

7 ： [  0. 110.  14.] [ 199.    0. -196.] 
 [[0.         0.12606271 0.16605057]
 [0

In [28]:
t_start = datetime.datetime.now()
h2_value3,lag_value3 = H2_whole_dichotomy2(signals3,0,12,1000,slideWindow=2,step=1,maxlag=0.2)
t_end = datetime.datetime.now()
print(t_end-t_start)

0:01:08.440878


In [23]:
print_h2_lag(h2_value3,lag_value3)

1 ： [   0. -190.  -20.] [190.   0. 150.] 
 [[0.         0.31846365 0.25939455]
 [0.32894879 0.         0.33884469]
 [0.19353317 0.30760651 0.        ]]

2 ： [   0.  -46. -199.] [  30.    0. -199.] 
 [[0.         0.38352    0.2530617 ]
 [0.37726124 0.         0.59168956]
 [0.23374683 0.67084149 0.        ]]

3 ： [   0.  -90. -166.] [ 130.    0. -199.] 
 [[0.         0.3111908  0.23718602]
 [0.34930989 0.         0.58463159]
 [0.26544084 0.60215684 0.        ]]

4 ： [   0. -110.   20.] [-100.    0.    0.] 
 [[0.         0.19811423 0.162487  ]
 [0.2063928  0.         0.39980622]
 [0.17110855 0.40066686 0.        ]]

5 ： [  0. -20. -10.] [190.   0.   0.] 
 [[0.         0.17348006 0.33169454]
 [0.14632529 0.         0.6113036 ]
 [0.18691541 0.60543047 0.        ]]

6 ： [ 0. 60. 90.] [108.   0.   0.] 
 [[0.         0.22262534 0.37924723]
 [0.11265209 0.         0.67042045]
 [0.2943878  0.61163598 0.        ]]

7 ： [  0. 110.  14.] [ 199.    0. -196.] 
 [[0.         0.12606271 0.16605057]
 [0

In [29]:
import scipy.io as sio

In [30]:
sio.savemat("data0324.mat",{"method":"h2","windowSize":2,"step":1,"maxLag":0.1,'electrode_names':['C45','C56','C66'],\
                           "start":0,"duration":12,"samplingRate":1000,'time_s':range(0,10),\
                           "h2":h2_value3,"lag":lag_value3})

In [31]:
read_data = sio.loadmat("data0324.mat")

In [32]:
read_data

{'__header__': b'MATLAB 5.0 MAT-file Platform: nt, Created on: Sun Mar 24 16:00:04 2019',
 '__version__': '1.0',
 '__globals__': [],
 'method': array(['h2'], dtype='<U2'),
 'windowSize': array([[2]]),
 'step': array([[1]]),
 'maxLag': array([[0.1]]),
 'electrode_names': array(['C45', 'C56', 'C66'], dtype='<U3'),
 'start': array([[0]]),
 'duration': array([[12]]),
 'samplingRate': array([[1000]]),
 'time_s': array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]),
 'h2': array([[[0.        , 0.31846365, 0.25939455],
         [0.32894879, 0.        , 0.33884469],
         [0.19353317, 0.30760651, 0.        ]],
 
        [[0.        , 0.38352   , 0.2530617 ],
         [0.37726124, 0.        , 0.59168956],
         [0.23374683, 0.67084149, 0.        ]],
 
        [[0.        , 0.3111908 , 0.23718602],
         [0.34930989, 0.        , 0.58463159],
         [0.26544084, 0.60215684, 0.        ]],
 
        [[0.        , 0.19811423, 0.162487  ],
         [0.2063928 , 0.        , 0.39980622],
         [0.1711

In [34]:
read_data["h2"].shape

(10, 3, 3)